In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
**Quickstart** ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
`Build Model <buildmodel_tutorial.html>`_ ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Quickstart
===================
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

Working with data
-----------------
PyTorch has two `primitives to work with data <https://pytorch.org/docs/stable/data.html>`_:
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.




In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torch.nn.functional as F


PyTorch offers domain-specific libraries such as `TorchText <https://pytorch.org/text/stable/index.html>`_,
`TorchVision <https://pytorch.org/vision/stable/index.html>`_, and `TorchAudio <https://pytorch.org/audio/stable/index.html>`_,
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO (`full list here <https://pytorch.org/vision/stable/datasets.html>`_). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [3]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


Read more about `loading data in PyTorch <data_tutorial.html>`_.




--------------




Creating Models
------------------
To define a neural network in PyTorch, we create a class that inherits
from `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU if available.



Read more about `building neural networks in PyTorch <buildmodel_tutorial.html>`_.




--------------




In [5]:
# define the network
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

import torch
class NormalNet(nn.Module):
    weights:list#[torch.Tensor]
    acts:list#[torch.nn.ReLU]
    biases:list#[torch.Tensor]
    num_layers:int 

    def __init__(self,input_dim:int,output_dim:int,layer_width:int,num_layers:int):
        self.num_layers = num_layers
        W = torch.rand(input_dim,layer_width)
        W = W/torch.norm(W)
        self.weights:list = [W]
        self.biases:list = [torch.zeros(layer_width)]
        self.acts:list=[torch.nn.ReLU()]
        for i in range(1,num_layers-1):
            W = torch.rand(layer_width,layer_width)
            W = W/torch.norm(W)
            self.weights.append(W)
            self.biases.append(torch.zeros(layer_width))
            self.acts.append(torch.nn.ReLU())
        W = torch.rand(layer_width,output_dim)
        W = W/torch.norm(W)
        self.weights.append(W)
        self.biases.append(torch.zeros(output_dim))
        self.acts.append(torch.nn.Identity()) # last layer with identity output
        
        with torch.no_grad():
            for i in range(0,self.num_layers):
                # mark for auto differentiation tape
                self.weights[i].requires_grad = True
                self.biases[i].requires_grad = True
        return None

    def forward_pass(self,input_tensor:torch.Tensor)->torch.Tensor:
        z =  input_tensor
        #self.acts[0](torch.matmul(input_tensor,self.weights[0]) +self.biases[0])
        #self.weights[0].requires_grad = True
        #self.biases[0].requires_grad = True
        for i in range(0,self.num_layers):
            self.weights[i].requires_grad = True
            self.biases[i].requires_grad = True
            z = self.acts[i](torch.matmul(z,self.weights[i]) +self.biases[i])
        return z
    
    def update_step(self,stepsize:float=1e-3):
        ## assumes loss.backward() is called
        with torch.no_grad():
            for i in range(0,self.num_layers):
                self.weights[i]=self.weights[i]- stepsize* self.weights[i].grad
                self.biases[i]=self.biases[i]-stepsize* self.biases[i].grad
        return None
    
    def clear_grads(self):
        #print("Clear Grads")
        for i in range(0,self.num_layers):
            if self.weights[i].grad is not None:
                self.weights[i].grad.data.zero_()
            if self.biases[i].grad is not None:
                self.biases[i].grad.data.zero_()
           
        return None
    
    def print_weights(self):
        print("weights")
        for i in range(0,self.num_layers):
            print("layer "+ str(i))
            print(self.weights[i])
    
    def print_weights_grad(self):
        print("weights grad")
        for i in range(0,self.num_layers):
            print("layer "+ str(i))
            print(self.weights[i].grad)
            
    def print_biases(self):
        print("biases")
        for i in range(0,self.num_layers):
            print("layer "+ str(i))
            print(self.biases[i])
            
    def print_biases_grad(self):
        print("biases")
        for i in range(0,self.num_layers):
            print("layer "+ str(i))
            print(self.biases[i].grad)

## some sanity checks 
test_net = NormalNet(3,2,10,4)
x = torch.rand(5,3)
print(x.size())
y = test_net.forward_pass(x)
print(y)
print(y.size())





Using cpu device
torch.Size([5, 3])
tensor([[0.2355, 0.3397],
        [0.2018, 0.2895],
        [0.3089, 0.4439],
        [0.2257, 0.3246],
        [0.1079, 0.1550]], grad_fn=<AddBackward0>)
torch.Size([5, 2])


In [6]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.



We also check the model's performance against the test dataset to ensure it is learning.



In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            x = nn.Flatten()(X)

            pred = model.forward_pass(x)
            test_loss += F.nll_loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [8]:
# get loss function and optimizer
batch_size = 64
input_dim = 784
output_dim = 10
layer_width = 10
layer_num = 3
model = NormalNet(input_dim,output_dim,layer_width,layer_num)
#loss_fn =  torch.nn.functional.binary_cross_entropy_with_logits

# --- sanity checks ---
x = torch.rand(batch_size,input_dim)
y = torch.randint(0,2,(batch_size,)) #random labels
print(y.size())

out = model.forward_pass(x)
print(out.size())

loss = F.nll_loss(out, y)
print("loss")
print(loss.size())
print(loss)

torch.Size([64])
torch.Size([64, 10])
loss
torch.Size([])
tensor(-3.1546, grad_fn=<NllLossBackward0>)


In [11]:
def train(dataloader, model, curr_epoch):
    size = len(dataloader.dataset)
    #model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        x = nn.Flatten()(X)

        pred = model.forward_pass(x)
        print("Loss: " +str(pred))
        loss = F.nll_loss(pred, y)
        print("Loss: " +str(loss))

        # Backpropagation
        #optimizer.zero_grad()
        loss.backward()
        #model.print_weights_grad()
        #model.print_biases_grad()
        #model.print_weights()
        #model.print_biases()
        #print("_____________")
        #
        model.update_step(stepsize=1e-3)
      
        model.clear_grads()
        #model.print_weights_grad()
        #model.print_biases_grad()

        #if batch % 100 == 0:
        #    loss, current = loss.item(), batch * len(X)
        #    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, t+1)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
Loss: tensor(-0.7304, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7669, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7154, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.8188, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7002, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7375, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7223, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7548, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7870, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.6941, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.8133, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.8285, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7840, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.8631, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7971, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7649, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7123, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.7371, grad_fn=<NllLossBackward0>)
Loss: tensor(-0.8529, grad_fn=<NllLossBackward0>)
Loss: tens

Loss: tensor(-2.2493, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.2702, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.3350, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.3037, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.4463, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.3597, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.2980, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.0743, grad_fn=<NllLossBackward0>)
Loss: tensor(-1.9671, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.3628, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.6200, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.6464, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.8965, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.5391, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.6754, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.4473, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.4747, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.5126, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.6870, grad_fn=<NllLossBackward0>)
Loss: tensor(-2.8941, grad_fn=<NllLossBackward0>)


Loss: tensor(-9.7941, grad_fn=<NllLossBackward0>)
Loss: tensor(-9.9636, grad_fn=<NllLossBackward0>)
Loss: tensor(-9.7414, grad_fn=<NllLossBackward0>)
Loss: tensor(-12.8140, grad_fn=<NllLossBackward0>)
Loss: tensor(-12.5716, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.9693, grad_fn=<NllLossBackward0>)
Loss: tensor(-12.0431, grad_fn=<NllLossBackward0>)
Loss: tensor(-10.4469, grad_fn=<NllLossBackward0>)
Loss: tensor(-10.8821, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.5930, grad_fn=<NllLossBackward0>)
Loss: tensor(-10.6309, grad_fn=<NllLossBackward0>)
Loss: tensor(-9.6331, grad_fn=<NllLossBackward0>)
Loss: tensor(-9.5362, grad_fn=<NllLossBackward0>)
Loss: tensor(-9.8987, grad_fn=<NllLossBackward0>)
Loss: tensor(-12.6278, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.8994, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.8063, grad_fn=<NllLossBackward0>)
Loss: tensor(-12.1109, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.4184, grad_fn=<NllLossBackward0>)
Loss: tensor(-11.6338, grad_fn=<NllLo

Loss: tensor(-200.5631, grad_fn=<NllLossBackward0>)
Loss: tensor(-199.1659, grad_fn=<NllLossBackward0>)
Loss: tensor(-246.3328, grad_fn=<NllLossBackward0>)
Loss: tensor(-255.5975, grad_fn=<NllLossBackward0>)
Loss: tensor(-250.4062, grad_fn=<NllLossBackward0>)
Loss: tensor(-254.8206, grad_fn=<NllLossBackward0>)
Loss: tensor(-256.3163, grad_fn=<NllLossBackward0>)
Loss: tensor(-241.7137, grad_fn=<NllLossBackward0>)
Loss: tensor(-299.4156, grad_fn=<NllLossBackward0>)
Loss: tensor(-300.4799, grad_fn=<NllLossBackward0>)
Loss: tensor(-292.7506, grad_fn=<NllLossBackward0>)
Loss: tensor(-292.1806, grad_fn=<NllLossBackward0>)
Loss: tensor(-294.9937, grad_fn=<NllLossBackward0>)
Loss: tensor(-312.7617, grad_fn=<NllLossBackward0>)
Loss: tensor(-296.8591, grad_fn=<NllLossBackward0>)
Loss: tensor(-338.1716, grad_fn=<NllLossBackward0>)
Loss: tensor(-382.1375, grad_fn=<NllLossBackward0>)
Loss: tensor(-342.8397, grad_fn=<NllLossBackward0>)
Loss: tensor(-374.1136, grad_fn=<NllLossBackward0>)
Loss: tensor

Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLossBackward0>)
Loss: tensor(nan, grad_fn=<NllLoss

AttributeError: 'NormalNet' object has no attribute '_modules'

Read more about `Training your model <optimization_tutorial.html>`_.




--------------




In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")